# The Labtime Class

The `Labtime` class is a tool for speeding up the simulation of process control experiments. With this tool you can more quickly develop control algorithms through simulation, then apply the algorithms to the Temperature Control Lab device with minimal changes to your code.

In most cases you do not need to directly invoke `Labtime`. For example, `setup` with the optional parameter `speedup` (described in the chapter _TCLab Simulation for Offline Use_) uses `Labtime` to adjust the operation of the `clock` iterator. This is sufficient for many applications.

## Basic Usage

### .time()

`Labtime` provides a replacement for the `time.time()` function from the Python standard library. The basic usage is demonstrated in the following cell. Note that `import` brings in an instance of `Labtime`. `labtime.time()` returns the _lab time_ elapsed since first imported into the Python kernal. 

In [1]:
from tclab import labtime

tic = labtime.time()
labtime.sleep(2)
toc = labtime.time()

print("Time since first imported = ", round(tic, 2), " labtime seconds.")
print("Time since first imported = ", round(toc, 2), " labtime seconds.")

Time since first imported =  0.0  labtime seconds.
Time since first imported =  2.01  labtime seconds.


By default, `labtime.time()` progresses at the same rate at real time as measured by the Python `time` package. The following cell demonstrates the default correspondence of labtime and real time.

In [2]:
from tclab import labtime
import time

time_start = time.time()
labtime_start = labtime.time()

def do(n):
    for k in range(0,n):
        t_real = time.time() - time_start
        t_lab  = labtime.time() - labtime_start
        print("real time = {0:4.2f}    lab time = {1:4.2f}".format(t_real, t_lab))
        time.sleep(1)
        
do(5)

real time = 0.00    lab time = 0.00
real time = 1.00    lab time = 1.00
real time = 2.01    lab time = 2.01
real time = 3.01    lab time = 3.01
real time = 4.01    lab time = 4.01


### .set_rate(rate) and .get_rate(rate)

Lab time can proceed at a rate faster or slower than real time. The relative rate of lab time to real time is set with the `labtime.set_rate(rate)`. The default value is one.  The current value of the rate is returned by the `get_rate()`.

In [3]:
from tclab import labtime
import time

time_start = time.time()
labtime_start = labtime.time()

labtime.set_rate(2)
print("Ratio of lab time to real time = ", labtime.get_rate())

do(5)

labtime.set_rate()
print("\nRatio of lab time to real time = ", labtime.get_rate())

do(5)

Ratio of lab time to real time =  2
real time = 0.00    lab time = 0.00
real time = 1.00    lab time = 2.00
real time = 2.01    lab time = 4.01
real time = 3.01    lab time = 6.02
real time = 4.01    lab time = 8.03

Ratio of lab time to real time =  1
real time = 5.02    lab time = 10.04
real time = 6.02    lab time = 11.04
real time = 7.02    lab time = 12.04
real time = 8.03    lab time = 13.05
real time = 9.03    lab time = 14.05


As demonstrated, conceptually you can thinkg of lab time as a piecewise linear function of real time with the following properties

* monotonically increasing
* continuous
* shared by all functions using `labtime`.

### .sleep(delay)

The `labtime.sleep()` function suspends execution for a period `delay` in lab time units. This is used, for example, in the `clock` iterator to speed up execution of a control loop when used in simulation mode.

## Advanced Usage

An additional set of functions are available in `Labtime` to facilitate construction of GUI's, and for programmatically creating code to simulate the behavior of more complex control systems.

### .reset(t)

The `labtime.reset(t)` method resets lab time to `t` (default 0). The function `setnow(t)` provides an equivalent service, and is included to provide backward compatibility early versions of `tclab`.  This function is typically  used within a GUI for repeated testing and tuning of a control algorithm.

In [4]:
from tclab import labtime

print("Resetting lab time to zero.")
labtime.reset(0)
print("labtime =", labtime.time(),"\n")

print("Resetting lab time to ten.")
labtime.reset(10)
print("labtime =", labtime.time(),"\n")

Resetting lab time to zero.
labtime = 3.2901763916015625e-05 

Resetting lab time to ten.
labtime = 10.000032901763916 



### .stop() / .start() /.running

`labtime.stop()` freezes the labtime clock at its current value. A Runtime warning is generated if there is an attempt to sleep while the labtime is stopped.

`labtime.start()` restarts the labtime clock following a stoppage.

`labtime.running` is a Boolean value that is `True` if the labtime clock is running, otherwise it is `False`.

In [5]:
from tclab import labtime
import time

print("Is labtime running?", labtime.running)
print("labtime =", labtime.time(), "\n")

print("Now we'll stop the labtime.")
labtime.stop()
print("Is labtime running?", labtime.running, "\n")

print("We'll pause for 2 seconds in real time.\n")
time.sleep(2)

print("We'll restart labtime and pick up where we left off.")
labtime.start()
print("labtime =", labtime.time())

Is labtime running? True
labtime = 10.015635967254639 

Now we'll stop the labtime.
Is labtime running? False 

We'll pause for 2 seconds in real time.

We'll restart labtime and pick up where we left off.
labtime = 10.015910148620605


## Auxiliary Functions

### clock(tperiod, tstep)

The `clock` iterator was introduced in an earlier section on synchronizing `tclab` with real time. In fact, `clock` uses the `Labtime` class to cooridinate with real time, and to provide faster than real time operation in simulation mode. 

In [6]:
from tclab import labtime, clock
import time

time_start = time.time()
labtime_start = labtime.time()

def do(n):
    print("\nRate =", labtime.get_rate())
    for t in clock(n):
        t_real = time.time() - time_start
        t_lab  = labtime.time() - labtime_start
        print("real time = {0:4.1f}    lab time = {1:4.1f}".format(t_real, t_lab))

labtime.set_rate(1)
do(5)

labtime.set_rate(10)
do(5)


Rate = 1
real time =  0.0    lab time =  0.0
real time =  1.0    lab time =  1.0
real time =  2.0    lab time =  2.0
real time =  3.0    lab time =  3.0
real time =  4.0    lab time =  4.0
real time =  5.0    lab time =  5.0

Rate = 10
real time =  5.0    lab time =  5.0
real time =  5.1    lab time =  6.0
real time =  5.2    lab time =  7.1
real time =  5.3    lab time =  8.0
real time =  5.4    lab time =  9.1
real time =  5.5    lab time = 10.0


### setnow(t)

`setnow(t)` performs the same function as `labtime.reset(t)`. This function appeared in an early version of `tclab`, and is included here for backwards compatibility.